#  比赛页面传送门： 常规赛：[遥感影像地块分割](https://aistudio.baidu.com/aistudio/competition/detail/63)
###  欢迎参加飞桨领航团实战速成营 一起学习 ，欢迎一键三连 Fork~
###  **赛题介绍：**
####  1、 本赛题由 2020 CCF BDCI 遥感影像地块分割 初赛赛题改编而来。遥感影像地块分割, 旨在对遥感影像进行像素级内容解析，对遥感影像中感兴趣的类别进行提取和分类，在城乡规划、防汛救灾等领域具有很高的实用价值，在工业界也受到了广泛关注。现有的遥感影像地块分割数据处理方法局限于特定的场景和特定的数据来源，且精度无法满足需求。因此在实际应用中，仍然大量依赖于人工处理，需要消耗大量的人力、物力、财力。本赛题旨在衡量遥感影像地块分割模型在多个类别（如建筑、耕地、林地等）上的效果，利用人工智能技术，对多来源、多场景的异构遥感影像数据进行充分挖掘，打造高效、实用的算法，提高遥感影像的分析提取能力。 赛题任务 本赛题旨在对遥感影像进行像素级内容解析，并对遥感影像中感兴趣的类别进行提取和分类，以衡量遥感影像地块分割模型在多个类别（如建筑、耕地、林地等）上的效果。

### 2、  数据说明
#### 本赛题提供了多个地区已脱敏的遥感影像数据，各参赛选手可以基于这些数据构建自己的地块分割模型。

### 训练数据集文件名称：train_and_label.zip

## 包含2个子文件，分别为：训练数据集（原始图片）文件、训练数据集（标注图片）文件，详细介绍如下：

* **训练数据集**（原始图片）文件名称：img_train

包含66,653张分辨率为2m/pixel，尺寸为256 * 256的JPG图片，每张图片的名称形如T000123.jpg。

* **训练数据集**（标注图片）文件名称：lab_train

包含66,653张分辨率为2m/pixel，尺寸为256 * 256的PNG图片，每张图片的名称形如T000123.png。

备注： 全部PNG图片共包括4种分类，像素值分别为0、1、2、3。此外，像素值255为未标注区域，表示对应区域的所属类别并不确定，在评测中也不会考虑这部分区域。

* **测试数据集**
测试数据集文件名称：img_test.zip，详细介绍如下：

包含4,609张分辨率为2m/pixel，尺寸为256 * 256的JPG图片，文件名称形如123.jpg。、

## 数据增强工具
**PaTTA:** 由第三方开发者组织AgentMaker维护的Test-Time Augmentation库，可在测试时通过数据增强方式产生额外的推理结果，在此基础上进行投票即可获得更稳定的成绩表现。 https://github.com/AgentMaker/PaTTA

**RIFLE:** 由第三方开发者对ICML 2020中的《RIFLE: Backpropagation in Depth for Deep Transfer Learning through Re-Initializing the Fully-connected LayEr》论文所提供的封装版本，其通过对输出层多次重新初始化来使得深层backbone得到更充分的更新。 https://github.com/GT-ZhangAcer/RIFLE_Module

In [5]:
!pip install paddlex -i https://mirror.baidu.com/pypi/simple
!pip install imgaug -i https://mirror.baidu.com/pypi/simple

Looking in indexes: https://mirror.baidu.com/pypi/simple
You should consider upgrading via the '/opt/conda/envs/python35-paddle120-env/bin/python -m pip install --upgrade pip' command.
Looking in indexes: https://mirror.baidu.com/pypi/simple
You should consider upgrading via the '/opt/conda/envs/python35-paddle120-env/bin/python -m pip install --upgrade pip' command.


In [6]:
# 导入包
import matplotlib
import os
import paddlex as pdx
import paddle.fluid as fluid # paddle深度学习框架包中的一个模块
import imgaug.augmenters as iaa
import numpy as np
# 设置使用0号GPU卡（如无GPU，执行此代码后仍然会使用CPU训练模型）
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/utils.py:26: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  def convert_to_list(value, n, name, dtype=np.int):


In [7]:
from paddlex.seg import transforms  # 语义分割
# 图像预处理和增强https://blog.csdn.net/weixin_43593330/article/details/107206239
train_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),   # 以一定概率对图像进行水平翻转，参数prob（float）：随机水平翻转概率，默认0.5
    transforms.Resize(target_size=256),  # 调整图像大小
    # transforms.RandomPaddingCrop(crop_size=256),  # 随机裁剪，当所需要的裁剪尺寸大于原图，则进行padding操作
    transforms.RandomBlur(prob=0.1),  # 以一定概率对图像进行高斯模糊，参数prob（float）：图像模糊概率，默认0.1
    transforms.RandomRotate(rotate_range=15),  # 对图像进行随机旋转，当存在标注图像时，同步进行，并对旋转后的图像进行padding
    # transforms.RandomDistort(brightness_range=0.5),  # 以一定概率对图像进行随机像素内容变换，该方法必须在Normalize之前使用
    transforms.Normalize()  # 归一化
])
eval_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.Normalize()
])

In [8]:
# 数据集的解压
# !unzip data/data81053/img_test.zip
# !unzip data/data81053/train_and_label.zip

In [9]:
datas = []
# 定义训练图片和标签
image_base = 'img_train'
annos_base = 'lab_train'
# os.listdir() 方法用于返回指定的文件夹包含的文件或文件夹的名字的列表。这个列表以字母顺序。 它不包括 '.' 和'..' 即使它在文件夹中
ids_ = [v.split('.')[0] for v in os.listdir(image_base)]

for id_ in ids_:
    img_pt0 = os.path.join(image_base, '{}.jpg'.format(id_))  # os.path.join() 方法把目录和文件名合成一个路径
    img_pt1 = os.path.join(annos_base, '{}.png'.format(id_))
    datas.append((img_pt0.replace('/home/aistudio/work/', ''), img_pt1.replace('/home/aistudio/work/', '')))  # 将 img_pt0和 img_pt1连接
    if os.path.exists(img_pt0) and os.path.exists(img_pt1):  # os.path.exists（）路径存在则返回True,路径损坏返回False
        pass
    else:
        raise "path invalid!"

print('total:', len(datas))
print(datas[0][0])
print(datas[0][1])

data_dir = '/home/aistudio/work/'

total: 66652
img_train/T094090.jpg
lab_train/T094090.png


In [10]:
# 给图像标签赋予含义
labels = [
    '建筑', '耕地', '林地',
    '其他'
]
# 标签写入labels.txt文件
with open('labels.txt', 'w') as f:
    for v in labels:
        f.write(v+'\n')
# 将数据进行打乱
np.random.seed(5)
np.random.shuffle(datas)
# 将数据划分为训练集和测试集
split_num = int(0.01*len(datas))

train_data = datas[:-split_num]
valid_data = datas[-split_num:]
# 将训练集写入train_list.txt
with open('train_list.txt', 'w') as f:
    for img, lbl in train_data:
        f.write(img + ' ' + lbl + '\n')
# 将测试集写入valid_list.txt
with open('valid_list.txt', 'w') as f:
    for img, lbl in valid_data:
        f.write(img + ' ' + lbl + '\n')

print('train:', len(train_data))
print('valid:', len(valid_data))

train: 65986
valid: 666


In [11]:
data_dir = './'
# paddlex.datasets的说明文档百度：https://paddlex.readthedocs.io/zh_CN/develop/apis/datasets.html  
# paddlex.datasets.SegDataset()用于语义分割模型
train_dataset = pdx.datasets.SegDataset(
    data_dir=data_dir,  # 数据集路径
    file_list='train_list.txt',  # 描述数据集图片文件和对应标注文件的文件路径
    label_list='labels.txt',  # 描述数据集包含的类别信息文件路径
    transforms=train_transforms,  # 数据集中每个样本的预处理/增强算子
    shuffle=True)  #是否需要对数据集中样本打乱顺序。默认为False
    
eval_dataset = pdx.datasets.SegDataset(
    data_dir=data_dir,
    file_list='valid_list.txt',
    label_list='labels.txt',
    transforms=eval_transforms)



2021-05-05 23:09:37 [INFO]	65986 samples in file train_list.txt
2021-05-05 23:09:37 [INFO]	666 samples in file valid_list.txt


In [12]:
num_classes_1 = len(train_dataset.labels)
regularizer_1=fluid.regularizer.L2DecayRegularizer(regularization_coeff=0.3)
# model=pdx.seg.HRNet(num_classes=num_classes_1, width=32)
model = pdx.seg.DeepLabv3p(
    num_classes=num_classes_1,  backbone='Xception65'
)


#file = open('./weights.txt', 'w')
# for v in model.trainable_variables:
#     file.write(str(v.name) + '\n')
#     file.write(str(v.shape) + '\n')
#     file.write(str(v.numpy()) + '\n')
# file.close()
model.train(
    num_epochs=40,  # 训练迭代轮数
    train_dataset=train_dataset,  # 训练数据读取器
    train_batch_size=16,  #训练数据batch大小，同时作为验证数据batch大小。默认32
    eval_dataset=eval_dataset,  # 评估数据读取器
    save_interval_epochs=2,  # 模型保存间隔，默认1
    log_interval_steps=200,  # 训练日志输出间隔，默认2
    save_dir='output/deeplab',  # 模型保存路径
    pretrain_weights='COCO',
    # optimizer = fluid.optimizer.Adam(learning_rate=4e-4, beta1=0.9, beta2=0.95,regularization=regularizer_1),
    # optimizer = fluid.optimizer.Adam(learning_rate=5e-4, beta1=0.9, beta2=0.95),
    optimizer = fluid.optimizer.MomentumOptimizer(learning_rate=3e-3, momentum=0.91, use_nesterov=True),
    lr_decay_power=0.925,
    use_vdl=True,
    #sensitivities_file='DEFAULT',
    eval_metric_loss=0.002,
    early_stop=True,
    early_stop_patience=4
    #resume_checkpoint='weights.txt'
)


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/math_op_patch.py:298: UserWarning: /opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddlex/cv/nets/xception.py:316
The behavior of expression A + B has been unified with elementwise_add(X, Y, axis=-1) from Paddle 2.0. If your code works well in the older versions but crashes in this version, try to use elementwise_add(X, Y, axis=0) instead of A + B. This transitional warning will be dropped in the future.
  op_type, op_type, EXPRESSION_MAP[method_name]))
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/framework.py:687: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release

2021-05-05 23:09:44 [INFO]	Connecting PaddleHub server to get pretrain weights...
2021-05-05 23:09:49 [INFO]	Load pretrain weights from output/deeplab/pretrain/DeepLabv3p_Xception65_COCO.


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/executor.py:1150: UserWarning: There are no operators in the program to be executed. If you pass Program manually, please use fluid.program_guard to ensure the current Program is being used.
  warnings.warn(error_info)


2021-05-05 23:09:49 [WARNING]	There is no pretrain weights loaded, maybe you should check you pretrain model!
2021-05-05 23:10:55 [INFO]	[TRAIN] Epoch=1/40, Step=200/4124, loss=1.56313, time_each_step=0.3s, eta=13:46:48
2021-05-05 23:11:56 [INFO]	[TRAIN] Epoch=1/40, Step=400/4124, loss=1.414656, time_each_step=0.3s, eta=13:58:37
2021-05-05 23:12:56 [INFO]	[TRAIN] Epoch=1/40, Step=600/4124, loss=1.291021, time_each_step=0.3s, eta=13:57:17
2021-05-05 23:13:57 [INFO]	[TRAIN] Epoch=1/40, Step=800/4124, loss=1.242409, time_each_step=0.31s, eta=14:0:47
2021-05-05 23:14:57 [INFO]	[TRAIN] Epoch=1/40, Step=1000/4124, loss=1.438219, time_each_step=0.3s, eta=13:53:46
2021-05-05 23:15:58 [INFO]	[TRAIN] Epoch=1/40, Step=1200/4124, loss=1.559485, time_each_step=0.3s, eta=13:49:25
2021-05-05 23:16:59 [INFO]	[TRAIN] Epoch=1/40, Step=1400/4124, loss=1.135852, time_each_step=0.3s, eta=13:48:7
2021-05-05 23:18:00 [INFO]	[TRAIN] Epoch=1/40, Step=1600/4124, loss=0.734391, time_each_step=0.31s, eta=14:5:32


100%|██████████| 42/42 [00:07<00:00,  6.00it/s]


...

2021-05-06 11:48:51 [INFO]	[EVAL] Finished, Epoch=36, miou=0.498456, category_iou=[0.48435636 0.70345939 0.49300916 0.31299875], oacc=0.672922, category_acc=[0.64155355 0.85793321 0.61340687 0.49892167], kappa=0.55515, category_F1-score=[0.65261466 0.82591859 0.66042349 0.4767693 ] .
2021-05-06 11:48:55 [INFO]	Model saved in output/deeplab/best_model.
2021-05-06 11:48:59 [INFO]	Model saved in output/deeplab/epoch_36.
2021-05-06 11:48:59 [INFO]	Current evaluated best model in eval_dataset is epoch_36, miou=0.4984559141305472
2021-05-06 11:49:44 [INFO]	[TRAIN] Epoch=37/40, Step=136/4124, loss=0.554248, time_each_step=0.31s, eta=1:23:47
2021-05-06 11:50:45 [INFO]	[TRAIN] Epoch=37/40, Step=336/4124, loss=0.630592, time_each_step=0.3s, eta=1:22:27
2021-05-06 11:51:46 [INFO]	[TRAIN] Epoch=37/40, Step=536/4124, loss=0.435939, time_each_step=0.3s, eta=1:21:27
2021-05-06 11:52:46 [INFO]	[TRAIN] Epoch=37/40, Step=736/4124, loss=0.681599, time_each_step=0.3s, eta=1:20:37
2021-05-06 11:53:47 [INFO

100%|██████████| 42/42 [00:07<00:00,  5.90it/s]


2021-05-06 12:31:04 [INFO]	[EVAL] Finished, Epoch=38, miou=0.510421, category_iou=[0.52454689 0.70391077 0.49183056 0.32139632], oacc=0.695356, category_acc=[0.68280083 0.79610535 0.5844427  0.6497901 ], kappa=0.583336, category_F1-score=[0.6881348  0.82622962 0.65936518 0.48644954] .
2021-05-06 12:31:08 [INFO]	Model saved in output/deeplab/best_model.
2021-05-06 12:31:11 [INFO]	Model saved in output/deeplab/epoch_38.
2021-05-06 12:31:11 [INFO]	Current evaluated best model in eval_dataset is epoch_38, miou=0.510421133024724
2021-05-06 12:31:42 [INFO]	[TRAIN] Epoch=39/40, Step=88/4124, loss=0.288298, time_each_step=0.3s, eta=0:41:44
2021-05-06 12:32:43 [INFO]	[TRAIN] Epoch=39/40, Step=288/4124, loss=0.290946, time_each_step=0.31s, eta=0:40:43
2021-05-06 12:33:44 [INFO]	[TRAIN] Epoch=39/40, Step=488/4124, loss=0.613046, time_each_step=0.31s, eta=0:39:52
2021-05-06 12:34:44 [INFO]	[TRAIN] Epoch=39/40, Step=688/4124, loss=0.853886, time_each_step=0.3s, eta=0:38:38
2021-05-06 12:35:45 [INFO

100%|██████████| 42/42 [00:07<00:00,  5.89it/s]


2021-05-06 13:13:15 [INFO]	[EVAL] Finished, Epoch=40, miou=0.497667, category_iou=[0.53342848 0.67420472 0.46010928 0.32292643], oacc=0.682026, category_acc=[0.65099894 0.79667228 0.66992486 0.53847685], kappa=0.562318, category_F1-score=[0.69573311 0.80540296 0.63023952 0.48820014] .
2021-05-06 13:13:18 [INFO]	Model saved in output/deeplab/epoch_40.
2021-05-06 13:13:18 [INFO]	Current evaluated best model in eval_dataset is epoch_38, miou=0.510421133024724


In [13]:
model.evaluate(eval_dataset, batch_size=1, epoch_id=None, return_details=False)
# eval_dataset (paddlex.datasets): 评估数据读取器。
# batch_size (int): 评估时的batch大小。默认1。
# epoch_id (int): 当前评估模型所在的训练轮数。
# return_details (bool): 是否返回详细信息。默认False。

2021-05-06 13:13:18 [INFO]	Start to evaluating(total_samples=666, total_steps=666)...


100%|██████████| 666/666 [00:11<00:00, 55.87it/s]


OrderedDict([('miou', 0.4976673934748348),
             ('category_iou',
              array([0.53342848, 0.67420472, 0.46010952, 0.32292685])),
             ('oacc', 0.6820258281298639),
             ('category_acc',
              array([0.65099894, 0.79667228, 0.66992536, 0.53847717])),
             ('kappa', 0.5623185568700259),
             ('category_F1-score',
              array([0.69573311, 0.80540296, 0.63023974, 0.48820061]))])

In [14]:
# 保存模型
model = pdx.load_model('./output/deeplab/best_model')

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/math_op_patch.py:298: UserWarning: /opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddlex/cv/nets/xception.py:316
The behavior of expression A + B has been unified with elementwise_add(X, Y, axis=-1) from Paddle 2.0. If your code works well in the older versions but crashes in this version, try to use elementwise_add(X, Y, axis=0) instead of A + B. This transitional warning will be dropped in the future.
  op_type, op_type, EXPRESSION_MAP[method_name]))


2021-05-06 13:13:34 [INFO]	Model[DeepLabv3p] loaded.


In [15]:
from tqdm import tqdm  # tqdm 是一个快速，可扩展的Python进度条，可以在 Python 长循环中添加一个进度提示信息，用户只需要封装任意的迭代器 tqdm(iterator)。
import cv2

test_base = 'img_testA/'
out_base = 'ccf_baidu_remote_sense/result/'

if not os.path.exists(out_base):  # 判断out_base的路径是否存在
    os.makedirs(out_base)  # os.makedirs() 方法用于递归创建目录


for im in tqdm(os.listdir(test_base)):  # os.listdir()返回path指定的文件夹包含的文件或文件夹的名字的列表
    if not im.endswith('.jpg'):  # 判断文件名后缀
        continue
    pt = test_base + im
    #result = model.overlap_tile_predict(pt, tile_size=[512, 512], pad_size=[64, 64], batch_size=32, transforms=eval_transforms)
    result = model.predict(pt)  # 使用训练好的模型
    cv2.imwrite(out_base+im.replace('jpg', 'png'), result['label_map'])  #  cv2.imwrite()保存图像

100%|██████████| 4608/4608 [01:12<00:00, 63.76it/s]


请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 